# Завдання

In [8]:
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from scipy.stats import uniform
import numpy as np
import random

_Вправа 1: натренуйте модель дерева рішень та виконайте пошук гіперпараметрів для moons dataset._ \
a. Згенеруйте датасет, використовуючи функцію `make_moons(n_samples=10000, noise=0.4)`. \
b. Розбийте її на тренувальну та тестувальну частини, використовуючи функцію `train_test_split()`. \
c. Використайте пошук з крос-валідацією (`GridSearchCV`, `RandomizedSearchCV`) для пошуку гіперпараметрів моделі `DecisionTreeClassifier`. Зокрема, спробуйте різні значення для параметра `max_leaf_nodes`.

In [2]:
X, y = make_moons(n_samples=10000, noise=0.4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, shuffle=True)

In [3]:
X,y

(array([[ 0.42008038, -0.14532633],
        [-0.50419326,  0.15371189],
        [ 2.19064545,  0.68921626],
        ...,
        [-1.15973992,  0.9216216 ],
        [-0.03929891, -0.27565892],
        [ 1.50229531, -0.68748025]]),
 array([1, 1, 1, ..., 0, 1, 1]))

c. Використайте пошук з крос-валідацією (`GridSearchCV`, `RandomizedSearchCV`) для пошуку гіперпараметрів моделі `DecisionTreeClassifier`. Зокрема, спробуйте різні значення для параметра `max_leaf_nodes`.

# Grid Search

In [34]:
dt_clf = DecisionTreeClassifier(random_state=42)

param_grid = {'criterion': ['gini', 'entropy', 'log_loss'],  'splitter': ['best', 'random'],'max_depth': [None, 5, 6, 8, 10, 15],
'max_features': ['sqrt', 'log2'], 'max_leaf_nodes': [5, 10, 20, 50, 100, 200]}

grid_search = GridSearchCV(dt_clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 5, 6, 8, 10, 15],
                         'max_features': ['sqrt', 'log2'],
                         'max_leaf_nodes': [5, 10, 20, 50, 100, 200],
                         'splitter': ['best', 'random']},
             scoring='accuracy')

In [44]:
print("Best params: ", grid_search.best_params_)

best parameters found:  {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 20, 'splitter': 'best'}


In [65]:
dt_grid_pred = grid_search.best_estimator_.predict(X_test)

In [66]:
print("★ Acc score:", accuracy_score(y_test, dt_grid_pred))

★ Acc score: 0.855


In [67]:
print("★ Confusion matrix:", confusion_matrix(y_test, dt_grid_pred))

★ Confusion matrix: [[865 125]
 [165 845]]


In [48]:
print(f"★ F1 score:", f1_score(y_test, dt_grid_pred))

F1 score: 0.8535353535353535


# Random Search

In [74]:
random_param_dist = {'n_estimators': randint(10, 200), 'max_depth': randint(3, 20),  'min_samples_split': randint(2, 20),
'min_samples_leaf': randint(1, 10), 'max_features': ['auto', 'sqrt', 'log2'],  'bootstrap': [True, False]  }

In [75]:
from sklearn.metrics import accuracy_score
from scipy.stats import randint

random_param_dist = {'max_leaf_nodes': randint(2, 200)}
random_search = RandomizedSearchCV(dt_clf, random_param_dist, n_iter=20, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'max_leaf_nodes': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fcdd4a28ca0>},
                   random_state=42, scoring='accuracy')

In [71]:
dt_random_pred = random_search.best_estimator_.predict(X_test)

In [68]:
print("Best params: ", random_search.best_params_)

Best params:  {'max_leaf_nodes': 22}


In [62]:
print("★ Acc score:", accuracy_score(y_test, dt_random_pred))

★ Acc score: 0.867


In [63]:
print("★ Confusion matrix:", confusion_matrix(y_test, dt_random_pred))

★ Confusion matrix: [[832 158]
 [108 902]]


In [64]:
print(f"★ F1 score:", f1_score(y_test, dt_random_pred))

★ F1 score: 0.8714975845410629


After searching for best hyperparameters for the model, the results show that the best max_leaf_nodes number is 10 for both approaches. The other parameters differ.

In [61]:
print("☆ Grid Search CV best params:", grid_search.best_params_)
print("★ Grid Search CV best score:", grid_search.best_score_)
print("☆ Random Search CV best params:", random_search.best_params_)
print("★ Random Search CV best score:", random_search.best_score_)

☆ Grid Search CV best params: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 20, 'splitter': 'best'}
★ Grid Search CV best score: 0.852875
☆ Random Search CV best params: {'max_leaf_nodes': 22}
★ Random Search CV best score: 0.8563749999999999


Grid Search seems to be more resource-intensive, it offers a more complex search through the parameteres. it's more rigoros. On the other hand, Randomized Search provides faster results, but it might not explore as deep as GS. This highlights the trade-off between computational efficiency and search depth when tuning hyperparameters.
I like Randomized Search more, because it offers a good balance between speed and effectiveness.

_Вправа 2_ : \
a. Завантажте датасет MNIST та розбийте його на тренувальну, валідаційну та тестувальну частини (наприклад, 50 000 / 10 000 / 10
000). \
b. Натренуйте різні класифікаційні моделі (Random Forest classifier, Logistic Regression, SVM). \
c. Далі, об'єднайте їх за допомогою голосування. \
d. Натренуйте нову модель, використовуючи виходи попередніх моделей на валідаційній вибірці (це будуть нові ознаки і нова тренувальна вибірка для даної моделі). Протестуйте отриманий ланцюжок на тренувальний вибірці. Такий спосіб поєднання моделей називається стогуванням (stacking).  

In [11]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:50000]
y_train = y[:50000]
X_test = X[60000:]
y_test = y[60000:]

X_val = X[50000:60000]
y_val = y[50000:60000]

In [12]:
#checked data type of data structure
type(mnist)

sklearn.utils._bunch.Bunch

In [44]:
from sklearn.utils import Bunch
for key, value in mnist.items():
    print(f'Attribute: {key}, Type: {type(value)}')

Attribute: data, Type: <class 'pandas.core.frame.DataFrame'>
Attribute: target, Type: <class 'pandas.core.series.Series'>
Attribute: frame, Type: <class 'pandas.core.frame.DataFrame'>
Attribute: categories, Type: <class 'NoneType'>
Attribute: feature_names, Type: <class 'list'>
Attribute: target_names, Type: <class 'list'>
Attribute: DESCR, Type: <class 'str'>
Attribute: details, Type: <class 'dict'>
Attribute: url, Type: <class 'str'>


In [64]:
# here's where we take out X tvector for the task
data_df = mnist.data
print(data_df.head())

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
# checking if it's meant to be normalized (nope)
data_df.dtypes.unique()

array([dtype('int64')], dtype=object)

In [13]:
#adding scaler in advance
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Random Forest Classifier

In [14]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_val_pred = rf_clf.predict(X_val)

rf_accuracy = accuracy_score(y_val, y_val_pred)
print(f'Random Forest Acc: {rf_accuracy:.4f}')

Random Forest Accuracy: 0.9736


# Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(max_iter=1000, random_state=42)
lr_clf.fit(X_train, y_train)
y_val_pred = lr_clf.predict(X_val)

lr_accuracy = accuracy_score(y_val, y_val_pred)
print(f'Logistic Regression Acc: {lr_accuracy:.4f}')

Logistic Regression Accuracy: 0.9219


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Support Vector Machine (SVM)

In [16]:
from sklearn.svm import SVC

svm_clf = SVC(kernel='rbf', random_state=42)
svm_clf.fit(X_train, y_train)
svm_y_pred = svm_clf.predict(X_test)

svm_accuracy = accuracy_score(y_test, svm_y_pred)
print(f'SVM Acc: {svm_accuracy:.4f}')

SVM Accuracy: 0.9785


# Models union via Voting Classifier

In [17]:
# using original ensemble of 3 models we've just trained to create a VC
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[('rf', rf_clf),('lr', lr_clf),('svm', svm_clf)], voting='hard')
voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Voting Classifier Accuracy: 0.9753


In [19]:
y_val_pred = voting_clf.predict(X_val)
voting_accuracy = accuracy_score(y_val, y_val_pred)
print(f'Voting Classifier (hard voting) Validation Acc: {voting_accuracy:.4f}')

hard_pred_final = voting_clf.predict(X_test)
print("Voting Classifier Test Acc: ", accuracy_score(y_test, hard_pred_final))

Voting Classifier Validation Accuracy: 0.9753
VotingClassifier (hard voting) Test Accuracy:  0.972


VC achieved strong performance. This demonstrates that combining classifiers is a nice options for improving model reliability, especially in cases where  models have different strengths and weaknesses.As it doesn't rely on a second model to learn from the base models, it is less expected to overfitting, as I understood.



# Creating a new output for meta model based on the results 3 models returned

as meta-model is trained using the predictions given by the base models on the validation set.

In [22]:
# a func to reshape values for the Meta Mod.
def create_stacked_features(X, rf_clf, lr_clf, svm_clf, scaler=None, fit_scaler=False):
    rf_pred = rf_clf.predict_proba(X)
    lr_pred = lr_clf.predict_proba(X)
    svm_decision = svm_clf.decision_function(X)
    # in multidim task - taking max val
    if svm_decision.ndim > 1:
        svm_decision = svm_decision.max(axis=1)

    # dim changes
    if scaler:
        if fit_scaler:
            # transform for training
            svm_decision_scaled = scaler.fit_transform(svm_decision.reshape(-1, 1))
        else:
            # just transform
            svm_decision_scaled = scaler.transform(svm_decision.reshape(-1, 1))
    else:
        # if no scaler, return unscaled decision values
        svm_decision_scaled = svm_decision.reshape(-1, 1)

    return np.hstack((rf_pred, lr_pred, svm_decision_scaled))

X_train_stack = create_stacked_features(X_train, rf_clf, lr_clf, svm_clf, scaler, fit_scaler=True)
X_val_stack = create_stacked_features(X_val, rf_clf, lr_clf, svm_clf, scaler, fit_scaler=False)


In [23]:
meta_model = LogisticRegression(max_iter=1000, random_state=42)
meta_model.fit(X_val_stack, y_val)

LogisticRegression(max_iter=1000, random_state=42)

In [24]:
X_test_stack = create_stacked_features(X_test, rf_clf, lr_clf, svm_clf, scaler)
y_test_pred = meta_model.predict(X_test_stack)
stacking_test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'꩜ Stacking Meta Model Test Acc: {stacking_test_accuracy:.4f}')

Stacking Meta Model Test Acc: 0.9665


And finally saving the model

In [26]:
import pickle
with open('meta_model.pkl', 'wb') as file:
    pickle.dump(meta_model, file)

# Conlcusions

I had a huge problem with training time for SVM model, as it is way more complex compared to the others. It turned out that there wasn't a seemly difference between accuracy scores of 3 models i've got. On the contrary, Random Forest showed better performance being the quickest-trained model.
Then I've got a Voting models based on ensemble of these 3, and it ended up showing good accuracy result as well (the accuracy score somewhere in betweeen of AS of 3 est-models)
And then I used stacking method to train a meta-model. I decided to use less complex model for it as the previous performances didn`t show the neccessity of it.
Also i've checked the difference of voting and stucking doing this task: it's said that in voting, final prediction is made by aggregating the decisions of each classifier, by selecting the class that appears most frequently/by applying weights to predictions. In contrast, stacking combines the outputs by passing them through a meta-classifier, which makes the final decision. Stacking has its own advantages as wekl, like improved prediction performance because of it helps reduce both bias and variance in the final model.

Also I attached my work before the deadline, but it was a version without the final conclusion, so I desided to upload this one as well.